In [1]:
import os
import glob
import json
from tld import get_fld
import seaborn as sns
import pandas as pd
from tqdm import tqdm
from IPython.display import HTML

In [2]:
# Constants
DATA_DIR = '../crawl_data'
SRC_DIR = '../crawler_src'

In [3]:
# Load the datasets
os.chdir(DATA_DIR)

# Get desktop json files
data_json_desktop = glob.glob('*_desktop.json')

# Get mobile json files
data_json_mobile = glob.glob('*_mobile.json')

In [4]:
def init_data_object():
    return  {
        # Per url data
        'tranco_ranks': [],
        'page_load_times': [],
        'num_requests': [],
        'distinct_third_parties': [],
        'num_distinct_tracker_domains': [],
        'num_distinct_tracker_entities': [],
        
        # Global data
        'failures': {
            'timeout_failures': 0,
            'TLS_failures': 0,
            'consent_failures': 0
        },
        'third_party_counts': {},
        'third_party_tracker_counts': {},
        'third_party_tracker_entities': {},
        'uber_cookie': {
            'request_hostname': '',
            'website': '',
            'num_cookies': 0,
            'first_party': False
        },
        'longest_lifespan_cookies': [],
        'canvas_fingerprints': [],
        'tracker_redirect_combos': []
    }

In [5]:
def parse_stupid_blocklist_to_something_readable(file_path):
    url_list = {}
    
    with open(file_path, encoding='utf-8') as blocklist_file:
        blocklist = json.load(blocklist_file)
        
        for cat, entities in blocklist['categories'].items():
            for entity_list in entities:
                for entity, url_objects in entity_list.items():
                    for url, aliases in url_objects.items():
                        all_urls = [url]
                        all_urls += aliases
                        
                        if entity not in url_list:
                            url_list[entity] = []
                            
                        for u in all_urls:
                            
                            try:
                                url_list[entity].append(get_fld(u, fix_protocol=True))
                            except:
                                pass
                        
                        url_list[entity] = list(set(url_list[entity]))
    
    return url_list

In [6]:
def create_stats_object(json_files):
    data_object = init_data_object()
    
    for json_file in tqdm(json_files):
        with open(json_file, 'r', encoding='utf-8') as data_file:
            try:
                data = json.load(data_file)
            except:
                print(f'Error opening json file: {json_file}, skipping')
                data = {"website_domain": json_file.split('_')[0],
                        "rank": -1,
                        "crawl_mode": "desktop",
                        "post_pageload_url": None,
                        "pageload_start_ts": -1,
                        "pageload_end_ts": -1,
                        "consent_status": None,
                        "requests": [],
                        "load_time": -1,
                        "cookies": -1,
                        "canvas_image_data": None,
                        "failure_status": {
                            "timeout": True,
                            "TLS": None,
                            "consent": False
                        }
                    }
                
            # Append tranco rank
            data_object['tranco_ranks'].append(int(data['rank']))
            
            # Updata failure counts
            if data['failure_status']['timeout']:
                data_object['failures']['timeout_failures'] += 1
            if data['failure_status']['TLS'] != 'null':
                data_object['failures']['TLS_failures'] += 1
            if data['failure_status']['consent']:
                data_object['failures']['consent_failures'] += 1
                
            # Only proceed if there is no timeout
            if data['failure_status']['timeout']:
                data_object['num_requests'].append(None)
                data_object['page_load_times'].append(None)
                data_object['distinct_third_parties'].append(None)
                data_object['num_distinct_tracker_domains'].append(None)
                data_object['num_distinct_tracker_entities'].append(None)
                continue
            
            # Append page load time
            data_object['page_load_times'].append(data['load_time'])
            
            # Append number of requests
            data_object['num_requests'].append(len(data['requests']))
            
            # Append distinct third parties
            get_fld_websocket = lambda u: get_fld(u[6:], fix_protocol=True) if u.startswith('wss://') \
                else get_fld(u, fix_protocol=True)
            distinct_third_parties = set([ get_fld_websocket(d['request_url'])
                                           for d in data['requests']
                                         ])
            distinct_third_parties.remove(get_fld(data['website_domain'], fix_protocol=True))
            data_object['distinct_third_parties'].append(len(distinct_third_parties))
            
            # Append number of distinct tracker domains
            tracker_dict = parse_stupid_blocklist_to_something_readable(SRC_DIR + '/disconnectmeblocklist.json')
            distinct_tracker_domains = []
            for third_party_domain in distinct_third_parties:
                for _, domains in tracker_dict.items():
                    if third_party_domain in domains:
                        distinct_tracker_domains.append(third_party_domain)
            data_object['num_distinct_tracker_domains'].append(len(distinct_tracker_domains))
            
            # Append number of distinct tracker entities/companies
            distinct_tracker_entities = []
            with open(SRC_DIR + '/domain_map.json', encoding='utf-8') as domain_map_json_file:
                domain_map_json = json.load(domain_map_json_file)
                
                for tracker_domain in distinct_tracker_domains:
                    if tracker_domain in domain_map_json.keys():
                        distinct_tracker_entities.append(domain_map_json[tracker_domain]['entityName'])
                    else:
                        for tracker_entity, tracker_entity_domains in tracker_dict.items():
                            if tracker_domain in tracker_entity_domains:
                                distinct_tracker_entities.append(tracker_entity)
                                break
                
            distinct_tracker_entities = set(distinct_tracker_entities)
            data_object['num_distinct_tracker_entities'].append(len(distinct_tracker_entities))
            
            # Update third party reference counts
            for party in distinct_third_parties:
                if party in data_object['third_party_counts']:
                    data_object['third_party_counts'][party] += 1
                else:
                    data_object['third_party_counts'][party] = 1
            
            # Update third party tracker counts
            for tracker in distinct_tracker_domains:
                if tracker in data_object['third_party_tracker_counts']:
                    data_object['third_party_tracker_counts'][tracker] += 1
                else:
                    data_object['third_party_tracker_counts'][tracker] =1
            
            # Update third party tracker entities
            for entity in distinct_tracker_entities:
                if entity in data_object['third_party_tracker_entities']:
                    data_object['third_party_tracker_entities'][entity] += 1
                else:
                    data_object['third_party_tracker_entities'][entity] =1
                    
            # Update the uber cookie
            max_cookie_count = None
            request_url = None
            for request in data['requests']:
                if 'cookie' not in request['request_headers']:
                    continue
                    
                cookie_count = len(request['request_headers']['cookie'].split(';'))
                if max_cookie_count is None or cookie_count > max_cookie_count:
                    max_cookie_count = cookie_count
                    request_url = request['request_url']
            
            if max_cookie_count is not None and \
               max_cookie_count > data_object['uber_cookie']['num_cookies']:
                data_object['uber_cookie']['num_cookies'] = max_cookie_count
                data_object['uber_cookie']['request_hostname'] = get_fld(request_url, fix_protocol=True)
                data_object['uber_cookie']['website'] = data['website_domain']
                data_object['uber_cookie']['first_party'] = data_object['uber_cookie']['request_hostname'] == \
                                                            data_object['uber_cookie']['website']
                
            # Get longest lasting cookies
            cookie_ids = []
            for request in data['requests']:
                if 'cookie' not in request['request_headers']:
                    continue
                    
                cookies = request['request_headers']['cookie'].split(';')
                for cookie in cookies:
                    cookie_ids.append(cookie.split('=')[0])
            
            all_cookies = []
            for cookie in data['cookies']:
                if cookie['name'] in cookie_ids:
                    cookie_data = cookie.copy()
                    cookie_data['size'] = len(cookie_data['value'])
                    
                    if 'sameSite' not in cookie_data:
                        cookie_data['sameSite'] = None

                    if 'expiry' not in cookie_data: # Cookies without expiry exist
                        cookie_data['expiry'] = 999999999999999
                    all_cookies.append(cookie_data)
                    all_cookies.append(cookie_data)

                sort_alg = lambda c: c['expiry']
                all_cookies.sort(key=sort_alg, reverse=True)
            data_object['longest_lifespan_cookies'] += all_cookies[:3]
            data_object['longest_lifespan_cookies'].sort(key=sort_alg, reverse=True)
            data_object['longest_lifespan_cookies'] = data_object['longest_lifespan_cookies'][:3]
            
            # HTTP redirect pairs for tracker domains
            tracker_redirect_combos = []
            for request in data['requests']:
                if request['response_status_code'] >= 300 and \
                   request['response_status_code'] <= 399 and \
                   'location' in request['response_headers']:
                    origin_domain = get_fld(request['request_url'], fix_protocol=True)
                    if request['response_headers']['location'].startswith('http'): # TODO: verify this filters exactly all non-relative URLs
                        redirect_domain = get_fld(request['response_headers']['location'], fix_protocol=True)
                    else: # location can be relative URL
                        redirect_domain = origin_domain
                    if origin_domain != redirect_domain and \
                       (origin_domain in distinct_tracker_domains or \
                        redirect_domain in distinct_tracker_domains):
                        tracker_redirect_combos.append((origin_domain, redirect_domain))
            data_object['tracker_redirect_combos'] += list(set(tracker_redirect_combos))
            
            # Fingerprints
            for fingerprint in data['canvas_image_data']:
                fingerprint['website'] = get_fld(data['website_domain'], fix_protocol=True)
            data_object['canvas_fingerprints'] += data['canvas_image_data']
            
    return data_object

In [7]:
# Create the data objects
crawls = {
    'desktop': create_stats_object(data_json_desktop),
    'mobile': create_stats_object(data_json_mobile)
}

 27%|████████████████▌                                            | 130/477 [00:07<00:19, 17.71it/s]


TldDomainNotFound: Domain 118.89.204.198 didn't match any existing TLD name!

In [8]:
def get_url_df(crawl_data):
    slice_keys = ['tranco_ranks', 'page_load_times', 'num_requests', 'distinct_third_parties',
              'num_distinct_tracker_domains', 'num_distinct_tracker_entities']
    df_input = {k: v for k, v in crawl_data.items() if k in slice_keys}
    return pd.DataFrame(data=df_input)

df_desktop = get_url_df(crawls['desktop'])
df_mobile = get_url_df(crawls['mobile'])

NameError: name 'crawls' is not defined

In [9]:
#1
desktop_failures = {'Page load timeout': crawls['desktop']['failures']['timeout_failures'],
                    'TLS error': crawls['desktop']['failures']['TLS_failures'],
                    'Consent click error': crawls['desktop']['failures']['consent_failures']}

mobile_failures = {'Page load timeout': crawls['mobile']['failures']['timeout_failures'],
                    'TLS error': crawls['mobile']['failures']['TLS_failures'],
                    'Consent click error': crawls['mobile']['failures']['consent_failures']}

failures_table = pd.DataFrame({'Crawl-desktop': desktop_failures, 'Crawl-mobile': mobile_failures})
failures_table.columns.name = 'Error type'
print(failures_table)

NameError: name 'crawls' is not defined

In [ ]:
#2
def draw_boxplot(metric):
    box_df = pd.DataFrame(df_desktop, columns=[metric])
    box_df.rename(columns={metric:'Crawl-desktop'}, inplace=True)
    box_df = box_df.join(df_mobile[metric])
    box_df.rename(columns={metric:'Crawl-mobile'}, inplace=True)
    return sns.boxplot(data=pd.DataFrame(data=box_df, columns=['Crawl-desktop','Crawl-mobile']))

g_pageload = draw_boxplot('page_load_times')
g_requests = draw_boxplot('num_requests')
g_thirdparties = draw_boxplot('distinct_third_parties')
g_trackdomains = draw_boxplot('num_distinct_tracker_domains')
g_trackentities = draw_boxplot('num_distinct_tracker_entities')


In [ ]:
#4,5,6
def print_ten_most_prev_websites_table(desktop_counts, mobile_counts):
    table_desktop = pd.DataFrame({'Third-party desktop': desktop_counts.keys(), 'Nr of websites desktop': desktop_counts.values()}).sort_values(by=['Nr of websites desktop'],ascending=False).reset_index(drop=True)
    table_mobile = pd.DataFrame({'Third-party mobile': mobile_counts.keys(), 'Nr of websites mobile': mobile_counts.values()}).sort_values(by=['Nr of websites mobile'],ascending=False).reset_index(drop=True)

    table = pd.concat([table_desktop, table_mobile],axis=1)

    # Some settings to make the table prettier
    pd.set_option('display.precision', 0) # No decimals in count
    pd.set_option('expand_frame_repr', False)
    table.index += 1 # Start at index 1
    columns=[('Crawl-desktop','Third-party'),('Crawl-desktop','Number of websites'),('Crawl-mobile','Third-party'),('Crawl-mobile','Number of websites') ]
    table.columns=pd.MultiIndex.from_tuples(columns) # Add nested column names

    #Only print the first 10 rows
    print(table.head(10))

print_ten_most_prev_websites_table(crawls['desktop']['third_party_counts'], crawls['mobile']['third_party_counts']) #4
print()
print_ten_most_prev_websites_table(crawls['desktop']['third_party_tracker_counts'], crawls['mobile']['third_party_tracker_counts'])#5
print()
print_ten_most_prev_websites_table(crawls['desktop']['third_party_tracker_entities'], crawls['mobile']['third_party_tracker_entities']) #6

In [ ]:
#7
distinct_third_parties_desktop = pd.DataFrame( {'Tranco rank': crawls['desktop']['tranco_ranks'], 'Distinct third-party domains': crawls['desktop']['distinct_third_parties']})
distinct_third_parties_mobile = pd.DataFrame( {'Tranco rank': crawls['mobile']['tranco_ranks'], 'Distinct third-party domains': crawls['mobile']['distinct_third_parties']})

# TODO: remove NaN values

sns.lmplot(x='Tranco rank', y='Distinct third-party domains', data=distinct_third_parties_desktop)
sns.lmplot(x='Tranco rank', y='Distinct third-party domains', data=distinct_third_parties_mobile)

In [ ]:
#8
distinct_trackers_desktop = pd.DataFrame( {'Tranco rank': crawls['desktop']['tranco_ranks'], 'Distinct tracker domains': crawls['desktop']['num_distinct_tracker_domains']})
distinct_trackers_mobile = pd.DataFrame( {'Tranco rank': crawls['mobile']['tranco_ranks'], 'Distinct tracker domains': crawls['mobile']['num_distinct_tracker_domains']})

# TODO: remove NaN values

sns.lmplot(x='Tranco rank', y='Distinct tracker domains', data=distinct_trackers_desktop)
sns.lmplot(x='Tranco rank', y='Distinct tracker domains', data=distinct_trackers_mobile)

In [ ]:
#9
data_9 = []
for crawl, crawl_data in crawls.items():
    data_9.append([crawl] + list(crawl_data['uber_cookie'].values()))
    
df_9 = pd.DataFrame(data_9)
df_9.columns = ['crawl'] + list(crawls['desktop']['uber_cookie'].keys())
df_9

In [ ]:
#10
data_10 = []
for crawl, crawl_data in crawls.items():
    for cookie in crawl_data['longest_lifespan_cookies']:
        data_10.append([crawl] + list(cookie.values()))

df_10 = pd.DataFrame(data_10)
df_10.columns = ['crawl'] + list(crawls['desktop']['longest_lifespan_cookies'][0].keys())
df_10

In [ ]:
#11
data_11 = []
for crawl, crawl_data in crawls.items():
    for combo in crawl_data['tracker_redirect_combos']:
        data_11.append([crawl, combo])
        
df_11 = pd.DataFrame(data_11)
df_11.columns = ['crawl', 'combo']
df_11.groupby(['crawl','combo'])['combo'] \
     .count() \
     .reset_index(name="count") \
     .sort_values(['crawl', 'count'], ascending=False)

In [ ]:
#12
canvas_fingerprints = []
for crawl, crawl_data in crawls.items():
    for fingerprint in crawl_data['canvas_fingerprints']:
        canvas_fingerprints.append([crawl,
                                    fingerprint['website'],
                                    fingerprint['fingerprint_script_resource_url'],
                                    '<img src="' + DATA_DIR + '/' + fingerprint['canvas_fingerprint_image'] + '" />'
                                   ])
df_canvas_fingerprints = pd.DataFrame(canvas_fingerprints)
df_canvas_fingerprints.columns = ['Crawl', 'Website', 'Fingerprint script URL', 'Canvas image']
df_canvas_fingerprints = df_canvas_fingerprints.to_html(escape=False)
HTML(df_canvas_fingerprints)